## A Federated model for PM2.5 prediction with cross validation
This notebook creates federated models for the PM2.5 prediction in Beijing based on pm25_beijing.py

In [9]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_federated as tff

from datetime import datetime
from pm25_beijing import DataHandler, fed_model_fn, create_fed_lstm

# data base path:
DATA_PATH = "data/pollution-data/"
FEATURES_TO_USE = ["TEMP", "PRES", "DEWP", "RAIN", "WSPM", "wd", "month", "day", "hour"]
TIMESTEPS = 48 # How many steps the LSTM should take into account
NUM_REG_CLASSES = 3
TRAIN_SPLIT = 0.25
BUFFER_SIZE = 256

MODEL_PATH_PREFIX = "models/federated_cross_val/"
FEDERATED_TRAINING_ROUNDS = 160
REDUCE_LR_EVERY = 64
LOGFILE_PREFIX = 'tensorboard-logs/'
BATCH_SIZE = 192
lr_client = 0.1
lr_server = 1

In [2]:
# For Tensorboard use
import nest_asyncio
nest_asyncio.apply()
%reload_ext tensorboard
%tensorboard --logdir tensorboard-logs --port=6006

Launching TensorBoard...

In [3]:
data = DataHandler(DATA_PATH, features_to_use=FEATURES_TO_USE)
data.preprocess_data(minmax_features=FEATURES_TO_USE)
data.interpolate()

Recognized wd (wind direction) as feature. Create columns north, east, south and west automatically.
Creating multiple classes from wd (wind direction):


100%|██████████| 12/12 [00:29<00:00,  2.44s/it]


In [5]:
# Create the input data for the models
K=5
federated_train_data_k = [[] for _ in range(K)]
federated_test_data_k = [None for _ in range(K)]
federated_test_labels_k = [[] for _ in range(K)]
first_station = True
all_stations = data.station

for station in all_stations:

    _, labels = data.create_classes(NUM_REG_CLASSES, station=station)

    station_data = data.create_model_input(TIMESTEPS, data.data[station])

    # Remove the labels which have no corresponding datapoints
    x, _, x_l, _= data.train_test_split(station_data, labels, TIMESTEPS, test_split=0, shuffle_data=False, num_stations=1)

    data_gen = DataHandler.cross_validation_data(x, x_l, K)

    i = 0
    for train_data, train_labels, test_data, test_labels in data_gen:
        ds = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(train_data), tf.convert_to_tensor(train_labels)))
        federated_train_data_k[i].append(ds)

        if federated_test_data_k[i] is None:
            federated_test_data_k[i] = test_data
        else:
            federated_test_data_k[i] = np.concatenate((federated_test_data_k[i], test_data))

        federated_test_labels_k[i].append(test_labels)

        i = i + 1

Creating model input from ['TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM', 'month', 'day', 'hour', 'wd_N', 'wd_E', 'wd_S', 'wd_W']


100%|██████████| 35016/35016 [00:28<00:00, 1226.86it/s]
2023-01-19 17:44:20.479015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-19 17:44:20.479067: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-19 17:44:20.479097: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (svsram): /proc/driver/nvidia/version does not exist


Creating model input from ['TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM', 'month', 'day', 'hour', 'wd_N', 'wd_E', 'wd_S', 'wd_W']


100%|██████████| 35016/35016 [00:28<00:00, 1224.12it/s]


Creating model input from ['TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM', 'month', 'day', 'hour', 'wd_N', 'wd_E', 'wd_S', 'wd_W']


...

100%|██████████| 35016/35016 [00:28<00:00, 1214.44it/s]


Creating model input from ['TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM', 'month', 'day', 'hour', 'wd_N', 'wd_E', 'wd_S', 'wd_W']


100%|██████████| 35016/35016 [00:28<00:00, 1213.41it/s]


Creating model input from ['TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM', 'month', 'day', 'hour', 'wd_N', 'wd_E', 'wd_S', 'wd_W']


100%|██████████| 35016/35016 [00:28<00:00, 1220.26it/s]


In [6]:
test_ds_k = [tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(td),
                                                 tf.convert_to_tensor(pd.concat(tl))))
             for td, tl in zip(federated_test_data_k, federated_test_labels_k)]


In [7]:
# Create the federated lstm models
def create_lstm(input_values=TIMESTEPS, input_features=12, num_output_classes=3):
    """Creates and returns a lstm model ready to be used in the federated structure

    :param input_values: Number of values to take into account for prediction ("timesteps")
    :type input_values: int, optional
    :param input_features: Number of features to take into account for prediction
    :type input_features: int, optional
    :param num_output_classes: Number of classes to do the classification for
    :type num_output_classes: int, optional
    :return: The not yet compiled tensorflow model
    :rtype: keras.Model
    """
    model = keras.Sequential()
    model.add(layers.LSTM(6,
                          return_sequences=True,
                          input_shape=(input_values, input_features),
                          kernel_initializer=tf.keras.initializers.GlorotUniform(seed=1)
                          ))

    model.add(layers.Dropout(.3))
    model.add(layers.LSTM(4,
                          return_sequences=False,
                          kernel_initializer=tf.keras.initializers.GlorotUniform(seed=1)
                          ))

    model.add(layers.Dropout(.3))
    model.add(layers.Dense(num_output_classes,
                           kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=1)
                           ))

    return model

In [6]:
def fed_model_fn():
    """A function for TFF to create a model during federated learning and return it is the correct type.

    :return: The LSTM model to be used as federated models
    :rtype: tff.learning.model
    """
    # We _must_ create a new model here, and _not_ capture it from an external
    # scope. TFF will call this within different graph contexts.
    keras_model = create_lstm()
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=(tf.TensorSpec(shape=(None, 48, 12), dtype=tf.float64, name=None),
                    tf.TensorSpec(shape=(None, 3), dtype=tf.int64, name=None)),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            tf.keras.metrics.CategoricalAccuracy(),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ])

In [ ]:
# The actual training
for i in range(K):
    print(f'### starting k-validation round {i} of {K}...')

    infos = f"6-4-arch-K{i}-reduce-ev{REDUCE_LR_EVERY}-b-{BATCH_SIZE}-lstm-SGD{lr_client}-SGD{lr_server}-{len(data.features)}-features"
    logfile = f'{LOGFILE_PREFIX}{datetime.now()} {infos}'  # für tensorboard log-Dateien

    batched_federated_train_data = [ds.batch(BATCH_SIZE) for ds in federated_train_data_k[i]]
    batched_test_ds = test_ds_k[i].batch(BATCH_SIZE)

    tff.federated_computation(lambda: 'Initialized!')()
    print("init")
    iterative_process = tff.learning.algorithms.build_weighted_fed_avg_with_optimizer_schedule(
        fed_model_fn,
        client_learning_rate_fn=lambda round_n: lr_client/10**tf.math.floor(tf.divide(round_n, REDUCE_LR_EVERY)),
        client_optimizer_fn=lambda x: tf.keras.optimizers.SGD(learning_rate=x),
        server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=lr_server)
    )

    print(iterative_process.initialize.type_signature.formatted_representation())
    state = iterative_process.initialize()

    summary_writer = tf.summary.create_file_writer(logfile)
    with summary_writer.as_default():
        for round_num in range(1, FEDERATED_TRAINING_ROUNDS):

            result = iterative_process.next(state, batched_federated_train_data)
            state = result.state
            metrics = result.metrics
            for name, value in metrics['client_work']['train'].items():
                tf.summary.scalar(name, value, step=round_num)
            # Test resulting model
            model = create_fed_lstm()
            model_weights = iterative_process.get_model_weights(state)
            model_weights.assign_weights_to(model)
            model.compile(
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                optimizer='sgd',
                metrics=['accuracy', tf.keras.metrics.Precision(),
                         tf.keras.metrics.Recall()]
            )
            scores = model.evaluate(batched_test_ds, verbose=0)
            tf.summary.scalar('test_accuracy', scores[1], step=round_num)
            tf.summary.scalar('test_precision', scores[2], step=round_num)
            tf.summary.scalar('test_recall', scores[3], step=round_num)
            lr = lr_client/(10**(int(round_num/REDUCE_LR_EVERY)))
            tf.summary.scalar('z_learning_rate', lr, step=round_num)

    model = create_fed_lstm()
    model_weights = iterative_process.get_model_weights(state)
    model_weights.assign_weights_to(model)
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        optimizer='sgd',
        metrics=['accuracy', tf.keras.metrics.Precision(),
                 tf.keras.metrics.Recall()]
    )
    model.save(f"{MODEL_PATH_PREFIX}fed_cross_val-{infos}")
    print(f'\nFINISHED federated training, logfile: {logfile}, NOW: {datetime.now()}')

### starting k-validation round 0 of 5...
init
( -> <
  global_model_weights=<
    trainable=<
      float32[12,24],
      float32[6,24],
      float32[24],
      float32[6,16],
      float32[4,16],
      float32[16],
      float32[4,3],
      float32[3]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=int32,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64
  >
>@SERVER)


INFO:tensorflow:Assets written to: model-6-4-arch-K0-reduce-ev64-b-192-lstm-SGD0.1-SGD1-9-features/assets


INFO:tensorflow:Assets written to: model-6-4-arch-K0-reduce-ev64-b-192-lstm-SGD0.1-SGD1-9-features/assets



FINISHED federated training, logfile: tensorboard-logs/2023-01-19 17:49:48.407068 6-4-arch-K0-reduce-ev64-b-192-lstm-SGD0.1-SGD1-9-features, NOW: 2023-01-19 18:43:16.773217
### starting k-validation round 1 of 5...
init


[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found


...

INFO:tensorflow:Assets written to: model-6-4-arch-K1-reduce-ev64-b-192-lstm-SGD0.1-SGD1-9-features/assets



FINISHED federated training, logfile: tensorboard-logs/2023-01-19 18:43:16.773359 6-4-arch-K1-reduce-ev64-b-192-lstm-SGD0.1-SGD1-9-features, NOW: 2023-01-19 19:36:46.492968
### starting k-validation round 2 of 5...
init


[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found


( -> <
  global_model_weights=<
    trainable=<
      float32[12,24],
      float32[6,24],
      float32[24],
      float32[6,16],
      float32[4,16],
      float32[16],
      float32[4,3],
      float32[3]
    >,
    non_trainable=<>
  >,
  distributor=<>,
  client_work=int32,
  aggregator=<
    value_sum_process=<>,
    weight_sum_process=<>
  >,
  finalizer=<
    int64
  >
>@SERVER)


[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found
[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found
[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found
[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found
[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found
[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found
[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found
[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found
[libprotobuf INFO google/protobuf/util/message_differencer.cc:1419] Proto type 'tensorflow.GraphDef' not found
[